# Формирование признаков пользователей и постов
В этом ноутбуке выполняется получение данных из базы данных, извлечение из них значимых признаков и сохранение полученных признаков для дальнейшего использования. Изначально по условию задачи известно, что массивы пользователей и постов фиксированы и меняться не будут.  

In [1]:
import pandas as pd
import numpy as np
from dotenv import dotenv_values
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer

SAVE_RAW_DATA_TO_CSV = False
SAVE_DATA_TO_CSV = False
SAVE_DATA_TO_DB = False

## Формирование строки подключения к базе данных
Для формирования строки подключения к базе данных используется скрытый файл `.env`, содержащий переменные окружения с данными, необходимыми для подключения к базе данных.

In [2]:
env_dict = dotenv_values("../creds/.env")

server = env_dict.get("KC_STARTML_POSTGRES_SERVER", '')
port = env_dict.get("KC_STARTML_POSTGRES_PORT")
db_name = env_dict.get("KC_STARTML_POSTGRES_DB")
user = env_dict.get("KC_STARTML_POSTGRES_USER", '')
password = env_dict.get("KC_STARTML_POSTGRES_PASSWORD")

password = (':' + password) if password else ''
port = (':' + port) if port else ''
db_name = ('/' + db_name) if db_name else ''

connection_string = f"postgresql://{user}{password}@{server}{port}{db_name}"

Получение из параметров окружения персонализированных наименований таблиц для сохранения сформированных признаков.  

In [3]:
USERS_FEATURES_TABLE_NAME = env_dict.get("KC_STARTML_USERS_FEATURES_TABLE", '')
POSTS_FEATURES_TABLE_NAME = env_dict.get("KC_STARTML_POSTS_FEATURES_TABLE", '')

## Подготовка данных.

In [4]:
def cat_feature_encode(data: pd.DataFrame, feature_col_name: str, value_col_name: str) -> pd.Series:
    """
    Кодирование категориального признака с использованием 
    числового признака, как правило уникального идентификатора.
    В качестве результата возвращается pandas-серия, в которой 
    индекс это значение кодируемого категориального признака, 
    а значение - числовая величина соответствующая значению 
    категориального признака.
    
    Пример использования:
    >>> df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6], 
    ...                    'cat_feature': ['a', 'b', 'a', 
    ...                                    'c', 'b', 'a']})
    >>> print(df)
       id cat_feature
    0   1           a
    1   2           b
    2   3           a
    3   4           c
    4   5           b
    5   6           a    
    >>> cat_map = cat_feature_encode(df, 'cat_feature', 'id')
    >>> df["cat_feature"] = df["cat_feature"].map(cat_map)
    >>> print(df)
       id  cat_feature
    0   1     1.000000
    1   2     0.491228
    2   3     1.000000
    3   4     0.000000
    4   5     0.491228
    5   6     1.000000
        
    Parameters
    ----------
    data : pandas.DataFrame
        Данные содержащие категориальный и целевой признак
    feature_col_name : str
        Наименование категориального признака
    value_col_name: str
        Наименование целевого признака, с помощью которого
        будет кодироваться категориальный
 
    Returns
    -------
    pandas.Series
        Серия, в которой индекс это значение кодируемого 
        категориального признака, а значение это числовая 
        величина соответствующая значению категориального 
        признака.
    """
    df = data.groupby(feature_col_name).agg({value_col_name: ["count", "min"]})
    
    mms = MinMaxScaler()
    df[(value_col_name, "min")] = mms.fit_transform(df[[(value_col_name, "min")]])
    df["code"] = df[(value_col_name, "count")] * 10 + df[(value_col_name, "min")]
    
    mms = MinMaxScaler()
    df["code"] = mms.fit_transform(df[["code"]])
    return df["code"]

### Формирование признаков пользователей.

**Получение данных пользователей**  
Это данные о всех доступных пользователях.

In [5]:
df_user_data_src = pd.read_sql(
    """
    SELECT
        user_id,
        gender,
        age,
        CONCAT(country, ', ', city) AS country_and_city,
        exp_group,
        os,
        source
    FROM public.user_data 
    """,
    con=connection_string
)

print(df_user_data_src.shape)
df_user_data_src.head(3)

(163205, 7)


,user_id,gender,age,country_and_city,exp_group,os,source
0,200,1,34,"Russia, Degtyarsk",3,Android,ads
1,201,0,37,"Russia, Abakan",0,Android,ads
2,202,1,17,"Russia, Smolensk",4,Android,ads


__Кодирование признаков пользователей:__

In [6]:
df_user_data = df_user_data_src.copy()

df_user_data = pd.get_dummies(df_user_data, 
                              columns=["os", "source"], 
                              drop_first=True, 
                              prefix_sep="_")

city_map = cat_feature_encode(df_user_data, 
                              "country_and_city", "user_id")
df_user_data["country_and_city"] = df_user_data["country_and_city"].map(city_map)

df_user_data.head(3)

,user_id,gender,age,country_and_city,exp_group,os_iOS,source_organic
0,200,1,34,0.000869,3,0,0
1,201,0,37,0.010972,0,0,0
2,202,1,17,0.019796,4,0,0


### Формирование признаков постов. ###

**Получение данных постов**  
Это данные о всех доступных постах.

In [7]:
df_post_text_src = pd.read_sql(
    """
    SELECT
        post_id,
        text,
        topic
    FROM public.post_text_df
    """,
    con=connection_string
)

print(df_post_text_src.shape)
df_post_text_src.head(3)

(7023, 3)


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business


__Кодирование признаков постов:__

In [8]:
df_post_text = df_post_text_src.copy()

tfidf = TfidfVectorizer()
tfidf.fit(df_post_text["text"])

text_vect = tfidf.transform(df_post_text["text"])

df_post_text["tfidf_mean"] = text_vect.mean(axis=1)
df_post_text["tfidf_max"] = text_vect.max(axis=1).todense()

del text_vect
del tfidf

df_post_data = df_post_text.drop("text", axis=1)

scaler = StandardScaler()
df_post_data[["tfidf_mean", "tfidf_max"]] = scaler.fit_transform(df_post_data[["tfidf_mean", "tfidf_max"]])

topic_map = cat_feature_encode(df_post_data, "topic", "post_id")
df_post_data["topic"] = df_post_data["topic"].map(topic_map)

df_post_data.head(3)

,post_id,topic,tfidf_mean,tfidf_max
0,1,0.04743,0.627607,0.731115
1,2,0.04743,1.492879,-0.773927
2,3,0.04743,1.700723,-0.894502


## Сохранение данных

### Сохранение признаков постов.

In [9]:
if SAVE_DATA_TO_DB:
    df_post_data.to_sql(POSTS_FEATURES_TABLE_NAME, 
                        con=connection_string, 
                        if_exists="replace")

In [10]:
if SAVE_DATA_TO_CSV:
    df_post_data.to_csv("../data/post_features.csv", index=False)

In [11]:
if SAVE_RAW_DATA_TO_CSV:
    df_post_text_src.to_csv("../data/post_text.csv", index=False)

### Сохранение признаков пользователей.

In [12]:
if SAVE_DATA_TO_DB:
    df_user_data.to_sql(USERS_FEATURES_TABLE_NAME, 
                        con=connection_string, 
                        if_exists="replace")

In [13]:
if SAVE_DATA_TO_CSV:
    df_user_data.to_csv('../data/user_features.csv', index=False)

In [14]:
if SAVE_RAW_DATA_TO_CSV:
    df_user_data_src.to_csv('../data/user_data.csv', index=False)

## Чтение данных из базы данных

### Чтение из базы признаков постов.

In [15]:
df_post_data_from_db = pd.read_sql(f"SELECT * FROM {POSTS_FEATURES_TABLE_NAME}",
                                   index_col="index",
                                   con=connection_string)
df_post_data_from_db.head(3)

,post_id,topic,tfidf_mean,tfidf_max
index,,,,
0,1,0.04743,0.627607,0.731115
1,2,0.04743,1.492879,-0.773927
2,3,0.04743,1.700723,-0.894502


### Чтение из базы признаков пользователей.

In [16]:
df_user_data_from_db = pd.read_sql(f"SELECT * FROM {USERS_FEATURES_TABLE_NAME}", 
                                   index_col="index",
                                   con=connection_string)
df_user_data_from_db.head(3)

,user_id,gender,age,country_and_city,exp_group,os_iOS,source_organic
index,,,,,,,
0,200,1,34,0.000869,3,0,0
1,201,0,37,0.010972,0,0,0
2,202,1,17,0.019796,4,0,0


## Чтение данных об активности пользователей
Данные об активности пользователей не требуют дополнительной обработки. Однако этих данных очень много. Поэтому имеет смысл выбрать небольшой фрагмент этих данных, который будет использоваться для обучения модели.  
Для начала надо выяснить, сколько есть данных и за какой период.   

In [17]:
pd.read_sql("""
    SELECT COUNT(*) AS rows_count, 
        MIN(timestamp) AS min_timestamp, MAX(timestamp) AS max_timestamp 
    FROM public.feed_data WHERE action = 'view'""", con=connection_string)

,rows_count,min_timestamp,max_timestamp
0,68686455,2021-10-01 06:01:40,2021-12-29 23:51:06


Для обучения модели будут использоваться данные за первую неделю, т.е. до 08.10.2021.

In [18]:
%%time
df_feed_data = pd.read_sql(
    """
    SELECT
        timestamp,
        user_id,
        post_id,
        target
    FROM public.feed_data
    WHERE action = 'view'
        AND timestamp < '2021-10-08'
    """,
    con=connection_string
)

df_feed_data = df_feed_data.sort_values(["timestamp", "user_id"]).reset_index(drop=True)
print(df_feed_data.shape)
df_feed_data.head(3)

(5360381, 4)
CPU times: user 24 s, sys: 3.92 s, total: 27.9 s
Wall time: 1min 15s


,timestamp,user_id,post_id,target
0,2021-10-01 06:01:40,1859,1498,1
1,2021-10-01 06:01:40,8663,3837,1
2,2021-10-01 06:01:40,15471,2810,0


In [19]:
if SAVE_RAW_DATA_TO_CSV:
    df_feed_data.to_csv('../data/feed_data.csv', index=False)